# Set up NB

In [ ]:
# %gui qt
%matplotlib notebook

import sys
import os
import time
import logging
from importlib import reload

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import qcodes as qc

from qcodes.instrument.parameter import Parameter

from pytopo.mplplots import init_nb_plotting; reload(init_nb_plotting)
from pytopo.mplplots.init_nb_plotting import *
from pytopo.mplplots import plots as mplplots

from pytopo.qctools import instruments as instools; reload(instools)
from pytopo.qctools.instruments import create_inst, add2station
from pytopo.qctools.dataset2 import select_experiment
from pytopo.qctools.measurement import MeasurementExt

from pytopo.rf import alazar_detector; reload(alazar_detector)
from pytopo.rf.alazar_detector import AlazarDetector

from plottr import qcodes_dataset; reload(qcodes_dataset)
from plottr.qcodes_dataset import QcodesDatasetSubscriber

# Initialize

## global settings

In [ ]:
SAMPLE = 'Cq334'

qc.config['core']['db_location'] = r"d:/OneDrive/BF1/Data/experiments.db"

qc.dataset.database.initialise_database()

## Create instruments

### Physical instruments

In [ ]:
inst_list = []

from qcodes.instrument_drivers.QuTech.IVVI import IVVI
ivvi = create_inst(IVVI, "ivvi", "ASRL5::INSTR")
inst_list.append(ivvi)

from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
key1 = create_inst(Keysight_34465A, "key1", "TCPIP::169.254.4.62::INSTR", force_new_instance=True)
key2 = create_inst(Keysight_34465A, "key2", "TCPIP::169.254.4.61::INSTR", force_new_instance=True)
inst_list.append(key1)
inst_list.append(key2)

from qcodes.instrument_drivers.stanford_research.SR860 import SR860
lockin1 = create_inst(SR860, "lockin1", "GPIB0::3::INSTR")
inst_list.append(lockin1)

from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014
awg = instools.create_inst(Tektronix_AWG5014, 'awg', address='TCPIP0::169.254.183.196::inst0::INSTR')
inst_list.append(awg)

for i in range(1,16):
    ivvi.parameters['dac{}'.format(i)].set_step(2)
    ivvi.parameters['dac{}'.format(i)].set_delay(0.001)

ivvi.dac4.set_delay(0.001)
ivvi.dac4.set_step(20)

ivvi.dac7.set_delay(0.001)
ivvi.dac7.set_step(20)

station = qc.Station(*inst_list)

### Virtual instruments

#### IVVI Rack

In [ ]:
from pytopo import ivvi as ivvi_ext; reload(ivvi_ext)
from pytopo.ivvi import IVVISetup

SD_current_resistance = 12e3 + 10e3 + 3.4e3
SD_dac_multiplier = 1e-2
SD_lockin_volt_multiplier = 1e-4
SD_current_gain = 1e7

ivvi_setup = create_inst(IVVISetup, 'ivvi_setup', ivvi=ivvi, force_new_instance=True)
station = add2station(station, ivvi_setup)


ivvi_setup.add_dac_voltage_src('SD', 
                               dac_number=4, 
                               multiplier=SD_dac_multiplier)

ivvi_setup.add_voltage_src('SD_lockin', 
                           param=lockin1.amplitude, 
                           multiplier=SD_lockin_volt_multiplier)

ivvi_setup.add_current_meas('SD', 
                            voltage_param=key1.volt, 
                            gain=SD_current_gain)

ivvi_setup.add_conductance_meas('SD',
                                param=lockin1.X,
                                gain=SD_current_gain,
                                r_series = 13.4e3,
                                lockin_dV = ivvi_setup.SD_lockin_voltage())
                                
ivvi_setup.add_dac_voltage_src('T1', 
                               dac_number=9, 
                               multiplier=1)
ivvi_setup.add_dac_voltage_src('T2', 
                               dac_number=10, 
                               multiplier=1)
ivvi_setup.add_dac_voltage_src('T3', 
                               dac_number=8, 
                               multiplier=1)
ivvi_setup.add_dac_voltage_src('T4', 
                               dac_number=11, 
                               multiplier=1)
ivvi_setup.add_dac_voltage_src('R2', 
                               dac_number=7, 
                               multiplier=0.001)
ivvi_setup.add_dac_voltage_src('R1', 
                               dac_number=5, 
                               multiplier=0.01)
ivvi_setup.add_dac_voltage_src('P2', 
                               dac_number=12, 
                               multiplier=1)
ivvi_setup.add_dac_voltage_src('P1', 
                               dac_number=13, 
                               multiplier=1)
ivvi_setup.add_dac_voltage_src('G_fine', 
                               dac_number=16, 
                               multiplier=0.01)
ivvi_setup.add_dac_voltage_src('G_coarse', 
                               dac_number=15, 
                               multiplier=1)

ivvi_setup.SD_lockin_voltage(1e-5)

def print_IVVI_dac_list():
    for i in range(16):
        print('IVVI DAC #{} at {} mV'.format(i+1, ivvi.get('dac{}'.format(i+1))))

In [ ]:
ivvi_setup.R2_voltage(1e-3)

In [ ]:
ivvi.dac6(1000)

#### Test IVVI with sample sim

In [ ]:
voltage = np.linspace(-10e-3, 10e-3, 11)
current = np.zeros_like(voltage)
lockin_mag = np.zeros_like(voltage)
lockin_phase = np.zeros_like(voltage)

for i, v in enumerate(voltage):
    ivvi_setup.SD_voltage(v)
    current[i] = ivvi_setup.SD_current()
    z = lockin1.X() + 1j * lockin1.Y()
    lockin_mag[i] = np.abs(z)
    lockin_phase[i] = np.angle(z, deg=True)

In [ ]:
corr_bias = ivvi_setup.corrected_bias(
    current, 
    voltage, 
    SD_current_resistance
)

dIdV = ivvi_setup.corrected_dIdV(
    lockin_mag, 
    lockin_dV=1e-4, 
    current_gain=1e7,
)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(voltage, current, 'o')
ax.plot(corr_bias, current, 'o')
ax.plot(voltage, voltage/100e3)
ax.set_xlabel('Bias (V)')
ax.set_ylabel('Current (nA)')
ax.grid()

## Set save Defaults

In [ ]:
ivvi.set_dacs_zero()
# ivvi_setup.SD_lockin_voltage(1e-5)

# Double Dot Experiment

In [ ]:
ivvi_setup.SD_voltage(0)
ivvi_setup.T1_voltage(0)
ivvi_setup.T2_voltage(0)
ivvi_setup.T3_voltage(0)
ivvi_setup.T4_voltage(0)
ivvi_setup.R1_voltage(0)
ivvi_setup.R2_voltage(0)
ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)
ivvi_setup.G_fine_voltage(0)
ivvi_setup.G_coarse_voltage(0)

In [ ]:
print_IVVI_dac_list()

## Pinch-off curves

In [ ]:
voltages = np.linspace(0, -0.5, 101)

ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(21.2)
ivvi_setup.SD_voltage(1e-3)
sweep_direction = Parameter('sweep_direction', get_cmd=None, set_cmd=None)
exp = select_experiment('Pinchoff', SAMPLE)

gates = [ivvi_setup.T1_voltage] # [ivvi_setup.R2_voltage] # [ivvi_setup.T2_voltage, ivvi_setup.T3_voltage, ivvi_setup.T4_voltage]


for gate in gates:
    
    # set all gates to zero.
    for g in [
        ivvi_setup.T1_voltage,
        ivvi_setup.T2_voltage, 
        ivvi_setup.T3_voltage, 
        ivvi_setup.T4_voltage,
        ivvi_setup.R1_voltage,
        ivvi_setup.R2_voltage,
        ivvi_setup.G_fine_voltage,
    ]:
        g(0)
    
    # now meaure pinch-off (up and down) for each gate in gates
    meas = MeasurementExt(station, exp=exp,
                          soft_sweep_params=[
                              gate,
                              sweep_direction,
                          ],
                          soft_data_params=[
                              lockin1.X,
                              lockin1.Y,
                              ivvi_setup.SD_current,
                          ])

    meas.write_period = 1

    with meas.run() as datasaver:

        datasaver.dataset.subscribe(
            QcodesDatasetSubscriber(datasaver.dataset), 
            state=[], min_wait=0, min_count=1,
        )

        for direction in [0,1]:
            sweep_direction(direction)

            volt = voltages
            if direction:
                volt = voltages[::-1]

            for g in volt:
                gate(g)
                datasaver.add_result(*meas.get_result())

## Coulomb blockade

In [ ]:
T2_vals = np.linspace(-0.2, -0.4, 6)[:]
T3_vals = [-0.3] # np.linspace(-0.3, -0.5, 6)
PG_vals = np.linspace(0, 1e-1, 101)
PG = awg.ch1_offset # ivvi_setup.R2_voltage # _fine_voltage

ivvi_setup.R2_voltage(0)
ivvi_setup.G_fine_voltage(0)
ivvi_setup.T4_voltage(0)
ivvi_setup.SD_voltage(250e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(21.2)

exp = select_experiment('Coulomb_blockade_scan', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.T2_voltage,
#                           ivvi_setup.T3_voltage,
                          PG,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          lockin1.Y,
                          ivvi_setup.SD_current,
                      ])

meas.write_period = 1

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )
    
    for t2 in T2_vals:
        ivvi_setup.T2_voltage(t2)
        
        for t3 in T3_vals:
            ivvi_setup.T3_voltage(t3)
            
            for pg in PG_vals:
                PG(pg)
                
                datasaver.add_result(*meas.get_result())

In [ ]:
PG_vals = np.linspace(0, 1e-1, 101)
PG = awg.ch1_offset

ivvi_setup.R2_voltage(0)
ivvi_setup.G_fine_voltage(0)
ivvi_setup.T4_voltage(0)
ivvi_setup.T2_voltage(-0.3)
ivvi_setup.T3_voltage(-0.3)
ivvi_setup.SD_voltage(250e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(21.2)

sweep_direction = Parameter('sweep_direction', get_cmd=None, set_cmd=None)

exp = select_experiment('Coulomb_blockade_scan', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          sweep_direction,
                          PG,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          lockin1.Y,
                          ivvi_setup.SD_current,
                      ])

meas.write_period = 1

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )
    
    for i in range(10):
        sweep_direction(i)
        vvals = PG_vals.copy()
        if i%2 > 0:
            vvals = vvals[::-1]
        
        for pg in vvals:
            PG(pg)

            datasaver.add_result(*meas.get_result())

In [ ]:
ivvi_setup.R2_voltage(0.1)

## Diamond scan

In [ ]:
PG_vals = np.linspace(0, 0.5, 101)
PG = ivvi_setup.P1_voltage
bias_vals = np.linspace(-0.4e-3, 0.4e-3, 161)

ivvi_setup.T1_voltage(-0.3)
ivvi_setup.T2_voltage(-0.3)
ivvi_setup.T3_voltage(0)
ivvi_setup.T4_voltage(0)

ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)

ivvi_setup.G_fine_voltage(0)
ivvi_setup.SD_voltage(0)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(21.2)

exp = select_experiment('Diamond_scan', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.SD_voltage,
                          PG,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          lockin1.Y,
                          ivvi_setup.SD_current,
                      ])

meas.write_period = 1

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )
    
    for pg in PG_vals:
        PG(pg)
            
        for bias in bias_vals:
            ivvi_setup.SD_voltage(bias)

            datasaver.add_result(*meas.get_result())

## DC CSD 

In [ ]:
ivvi_setup.R1_voltage(0)
ivvi_setup.R2_voltage(0)
ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)

ivvi_setup.T1_voltage(-0.3)
ivvi_setup.T2_voltage(-0.3)
ivvi_setup.T3_voltage(-0.44)
ivvi_setup.T4_voltage(0)


# TG = ivvi_setup.T4_voltage
# TG_vals = np.linspace(-0.34,-0.37,4)

ivvi_setup.SD_voltage(0e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(21.2)

exp = select_experiment('DC_CSD_tryDD', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.P1_voltage,
                          ivvi_setup.G_fine_voltage,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          ivvi_setup.SD_current,         
                      ])

meas.write_period = 1

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )

#     for tg in TG_vals:
#         TG(tg)
        
#         print(time.strftime('%Y-%m-%d %H:%M:%S') + ' setting {} to {} V'.format(TG, tg))
    
    for p1 in np.linspace(0, 0.5, 51):
        ivvi_setup.P1_voltage(p1)

        for pg in np.linspace(0, 1e-3, 51):
            ivvi_setup.G_fine_voltage(pg)

            datasaver.add_result(*meas.get_result())

### with R1

In [ ]:
ivvi_setup.R1_voltage(0)
ivvi_setup.R2_voltage(0)
ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)

ivvi_setup.T1_voltage(-0.3)
ivvi_setup.T2_voltage(-0.3)
ivvi_setup.T3_voltage(-0.44)
ivvi_setup.T4_voltage(0)


# TG = ivvi_setup.T4_voltage
# TG_vals = np.linspace(-0.34,-0.37,4)

ivvi_setup.SD_voltage(0e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(21.2)

exp = select_experiment('DC_CSD_tryDD', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.R1_voltage,
                          ivvi_setup.G_fine_voltage,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          ivvi_setup.SD_current,         
                      ])

meas.write_period = 1

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )

#     for tg in TG_vals:
#         TG(tg)
        
#         print(time.strftime('%Y-%m-%d %H:%M:%S') + ' setting {} to {} V'.format(TG, tg))
    
    for r1 in np.linspace(0, 2.5e-3, 51):
        ivvi_setup.R1_voltage(r1)

        for pg in np.linspace(0, 1e-3, 51):
            ivvi_setup.G_fine_voltage(pg)

            datasaver.add_result(*meas.get_result())

### with R2

In [ ]:
ivvi_setup.T3_voltage()

In [ ]:
ivvi_setup.R1_voltage(0)
ivvi_setup.R2_voltage(0)
ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)

ivvi_setup.T1_voltage(0)
ivvi_setup.T2_voltage(-0.25)
# ivvi_setup.T3_voltage(-0.45)
ivvi_setup.T4_voltage(-0.25)

ivvi_setup.SD_voltage(0e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(16.2)

exp = select_experiment('DC_CSD_tryDD', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.R2_voltage,
                          ivvi_setup.G_fine_voltage,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          ivvi_setup.SD_current,
                          ivvi_setup.SD_conductance,
                      ])

meas.write_period = 1

TG_vals = [-0.37826086956521743]

for tg in TG_vals:
    ivvi_setup.T3_voltage(tg)
    
    with meas.run() as datasaver:

        datasaver.dataset.subscribe(
            QcodesDatasetSubscriber(datasaver.dataset), 
            state=[], min_wait=0, min_count=1,
        )

#     print(time.strftime('%Y-%m-%d %H:%M:%S') + ' setting {} to {} V'.format(TG, tg))

        for r2 in np.linspace(-1e-3, 1e-3, 25):
            ivvi_setup.R2_voltage(r2)

            for pg in np.linspace(0e-3, 2e-3, 51):
                ivvi_setup.G_fine_voltage(pg)

                datasaver.add_result(*meas.get_result())

In [ ]:
ivvi_setup.R1_voltage(0)
ivvi_setup.R2_voltage(0)
ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)

ivvi_setup.T1_voltage(0)
ivvi_setup.T2_voltage(-0.29)
ivvi_setup.T3_voltage(-0.40)
ivvi_setup.T4_voltage(-0.29)

ivvi_setup.SD_voltage(0e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(16.2)

exp = select_experiment('DC_CSD_tryDD', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.R2_voltage,
                          ivvi_setup.G_fine_voltage,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          ivvi_setup.SD_current,
                          ivvi_setup.SD_conductance,
                      ])

meas.write_period = 1

TG_vals = np.linspace(-0.31,-0.26,12)

for tg in TG_vals:
    ivvi_setup.T2_voltage(tg)
    
    with meas.run() as datasaver:

        datasaver.dataset.subscribe(
            QcodesDatasetSubscriber(datasaver.dataset), 
            state=[], min_wait=0, min_count=1,
        )

#     print(time.strftime('%Y-%m-%d %H:%M:%S') + ' setting {} to {} V'.format(TG, tg))

        for r2 in np.linspace(-1e-3, 1e-3, 51):
            ivvi_setup.R2_voltage(r2)

            for pg in np.linspace(0e-3, 2e-3, 101):
                ivvi_setup.G_fine_voltage(pg)

                datasaver.add_result(*meas.get_result())

#### Bias scan

In [ ]:
ivvi_setup.R1_voltage(0)
ivvi_setup.R2_voltage(0)
ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)

ivvi_setup.T1_voltage(0)
ivvi_setup.T2_voltage(-0.29)
ivvi_setup.T3_voltage(-0.41)
ivvi_setup.T4_voltage(-0.29)


# TG = ivvi_setup.T4_voltage
# TG_vals = np.linspace(-0.34,-0.37,4)

ivvi_setup.SD_voltage(0e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(16.2)

exp = select_experiment('DC_CSD_tryDD', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.R2_voltage,
                          ivvi_setup.SD_voltage,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          ivvi_setup.SD_current,
                      ])

meas.write_period = 1

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )

#     for tg in TG_vals:
#         TG(tg)
        
#     print(time.strftime('%Y-%m-%d %H:%M:%S') + ' setting {} to {} V'.format(TG, tg))
    
    for r2 in np.linspace(0, 2e-3, 25):
        ivvi_setup.R2_voltage(r2)

        for bias in np.linspace(-0.5e-3, 0.5e-3, 51):
            ivvi_setup.SD_voltage(bias)

            datasaver.add_result(*meas.get_result())

### CSD tune lockin frequency

In [ ]:
ivvi_setup.R1_voltage(0)
ivvi_setup.R2_voltage(0)
ivvi_setup.P1_voltage(0)
ivvi_setup.P2_voltage(0)

ivvi_setup.T1_voltage(0)
ivvi_setup.T2_voltage(-0.3)
ivvi_setup.T3_voltage(-0.45)
ivvi_setup.T4_voltage(-0.3)


# TG = ivvi_setup.T4_voltage
# TG_vals = np.linspace(-0.34,-0.37,4)

ivvi_setup.SD_voltage(0e-6)
ivvi_setup.SD_lockin_voltage(10e-6)
lockin1.frequency(21.2)

exp = select_experiment('DC_CSD_tryDD', SAMPLE)

meas = MeasurementExt(station, exp=exp,
                      soft_sweep_params=[
                          ivvi_setup.G_fine_voltage,
                          lockin1.frequency,
                      ],
                      soft_data_params=[
                          lockin1.X,
                          ivvi_setup.SD_current,
   
                      ])

meas.write_period = 1

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )

#     for tg in TG_vals:
#         TG(tg)
        
#     print(time.strftime('%Y-%m-%d %H:%M:%S') + ' setting {} to {} V'.format(TG, tg))
    
    for freq in np.linspace(10,30,11):
        lockin1.frequency(freq)

        for pg in np.linspace(0, 1e-3, 51):
            ivvi_setup.G_fine_voltage(pg)

            datasaver.add_result(*meas.get_result())

In [ ]:
%%timeit
lockin1.X()

In [ ]:
key1.NPLC(10)

In [ ]:
%%timeit
key1.volt()

In [ ]:
key1.volt()